# ⭐ O colab abaixo, tem como objetivo realizar o treino de 2 modelos usando um conjunto com 10 classes, realizando o tunning dos paramestros para alcançar um bom resultado ⭐.

---


In [ ]:
## Realizando a importação das bibliotecas ###

from sklearn.datasets import fetch_20newsgroups # dataset de texto para classificação contendo 20 classes
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer # carrega Vectorizer e TFIDF
from sklearn.naive_bayes import MultinomialNB # algoritmo do Naive Bayes
from sklearn.pipeline import Pipeline # Cria pipeline contendo todas as transformações e modelo
from nltk.stem.snowball import SnowballStemmer # Função que retorna a palavra a sua raiz
import numpy as np
from sklearn.linear_model import SGDClassifier # Algoritmo Gradient Descendente Stocastico
from sklearn.model_selection import GridSearchCV
import nltk 
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
import warnings
import matplotlib.pyplot as plt 

warnings.simplefilter('ignore')


In [ ]:
# Carrega o dataset de treinamento do fetch 20 news groups
newsgroups = fetch_20newsgroups(subset='train') 

In [ ]:
## Lista de classes disponiveis para usar nos treinamentos ##
list(newsgroups.target_names)

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
## Realizado as escolhas das classes que serão usadas para os modelos, nota-se que peguei os modelos que sejam mais semalhantes para conseguir uma boa correlação ##
categories = ['comp.graphics',
              'comp.os.ms-windows.misc',
              'comp.sys.ibm.pc.hardware',
              'comp.sys.mac.hardware',              
              'comp.windows.x',
      #---------------------------------------------------------------#
      # ⇈ Os 5 modelos acima são relacionados a tecnlogia            #
      # ⇊ Os 5 modelos abaixo são relacionados a politica e religião #
      #---------------------------------------------------------------#
              'soc.religion.christian',
              'talk.politics.guns',
              'talk.politics.mideast',
              'talk.politics.misc',
              'talk.religion.misc']

#Após a escolha das classes será feito o treino e teste
df_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
df_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True)

In [ ]:
## Apresentando as classes escolhidas acima ##
df_train.target_names

['comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']



---


# **Iniciando a primeira abordagem**

#####Nesta primeira abordagem, vamos usar um modelo sem o tunning de parametros, e vamos coletar o resultado para depois fazer um comparativo com um modelo tunado 🚀#####


---

In [ ]:
# Pipeline de machine learning 
clf_1 = Pipeline([
    #1- Aplicar o count vectorizer nos textos, para converter em uma matriz e fazer a frequência de ocorrência de cada palavra.
    ('vect', CountVectorizer()),
    #2- Passo 2, aplicar o TFIDF nos textos levando em consideração a frequência com que aparecem em cada documento e sua raridade.
    ('tfidf', TfidfTransformer()),
    #3- aplicar o algoritmo Naive Bayes para calcular a probabilidade de que um objeto (por exemplo, um documento de texto) pertença a uma determinada classe
    ('clf', MultinomialNB())]) 

In [ ]:
 # Realiza o treinamento do modelo no pipeline
clf_trained = clf_1.fit(df_train.data, df_train.target)

In [ ]:
 # Faz predição no dado de teste
pred = clf_trained.predict(df_test.data)

In [ ]:
#   Abaixo será gerado as métricas de assertividade do modelo e a precisão de cada classe.
#   Conforme acima, tentamos pegar os 10 dados que teria uma correlação maior entre eles pegando 5 de cada tema
creport = classification_report(df_test.target, pred, target_names=df_test.target_names)
print(creport)

                          precision    recall  f1-score   support

           comp.graphics       0.83      0.72      0.77       389
 comp.os.ms-windows.misc       0.81      0.67      0.73       394
comp.sys.ibm.pc.hardware       0.74      0.85      0.79       392
   comp.sys.mac.hardware       0.88      0.83      0.85       385
          comp.windows.x       0.89      0.79      0.84       395
  soc.religion.christian       0.53      0.99      0.69       398
      talk.politics.guns       0.64      0.96      0.77       364
   talk.politics.mideast       0.95      0.92      0.93       376
      talk.politics.misc       0.97      0.42      0.59       310
      talk.religion.misc       1.00      0.15      0.26       251

                accuracy                           0.76      3654
               macro avg       0.82      0.73      0.72      3654
            weighted avg       0.81      0.76      0.74      3654



In [ ]:
## Será mostrado abaixo o resultado do acurácia sem o tunning de parametros ##
acc = np.mean(pred == df_test.target)
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.7567049808429118




---


# **Iniciando a SEGUNDA abordagem**

#####Agora vamos pegar os mesmos dados e realizar o tunning de parametros e vamos fazer o comparativo com a primeira abordagem #####

#####Para isto vamos usar o modelo **Naive Bayes** com **Grid Search**


# 🚀**Tunning de parametros**🚀


---

In [ ]:
# Grid Search
# Aqui, estamos criando uma lista de parâmetros para os quais gostaríamos de fazer o ajuste de desempenho.
# Por exemplo. vect__ngram_range; aqui estamos dizendo para usar unigramas e bigramas e escolher aquele que é o ideal.

parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}

In [ ]:
## O Tempo da execução pode levar entre 3 a 4 minutos pela quantidade de classses escolhidas.

 # Define o grid search para buscar os melhores parametros
gs_clf = GridSearchCV(clf_trained, parameters, n_jobs=-1) 

# treinamento do modelo 
gs_clf = gs_clf.fit(df_train.data, df_train.target) 

In [ ]:
# Para ver a melhor pontuação média e os parâmetros que foram aplicados, execute o seguinte código

print(gs_clf.best_score_)
gs_clf.best_params_

0.9205410350799414


{'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [ ]:
# Faz predição no dado de teste
pred2 = gs_clf.predict(df_test.data) 

In [ ]:
##RESULTADO##
acc2 = np.mean(pred2 == df_test.target)
print('>>>> Acurácia: ', acc2)

>>>> Acurácia:  0.8229337712096333


In [ ]:
#Fazendo o comparativo dos dois modelos

#SEM TUNNING DE PARAMETROS
print('>>>> Acurácia SEM tunning: ', acc)

print('\n')

#COM TUNNING DE PARAMETROS
print('>>>> Acurácia COM tunning: ', acc2)

#Resumimos que um conjunto de dados, mesmo que seja com dados diferentes, conseguimos através do Tunning, um bom resultado de Acurácia!!

>>>> Acurácia SEM tunning:  0.7567049808429118


>>>> Acurácia COM tunning:  0.8229337712096333
